In [ ]:
import enum
import cv2
import traitlets
import ipywidgets.widgets as widgets
import numpy as np
import time
import glob

from camera import Camera, bgr8_to_jpeg

# Collect data of checkerboard

In [ ]:
cameraL = Camera.instance(id=0, width=820, height=616)
cameraR = Camera.instance(id=1, width=820, height=616)

In [ ]:
wL=widgets.Image(width=410, height=308)
wR=widgets.Image(width=410, height=308)
display(widgets.HBox([wL,wR]))

In [ ]:
# Set the path to the images captured by the left and right cameras
pathL = "./data/stereoL/"
pathR = "./data/stereoR/"

# Checkerboard size
cx=8
cy=5

# Time countdown before taking image
T = 10

Taking pictures for calibration ..

In [ ]:
start = time.time()
count = 0

try:
    while True:
        timer = T - int(time.time() - start)
        frameR= cameraR.value
        frameL= cameraL.value

        img1_temp = frameL.copy()
        status=f"{timer} - found {count}"
        cv2.putText(img1_temp,status,(50,75),1,4,(0,255,255),3)
        wR.value=bgr8_to_jpeg(frameR)
        wL.value=bgr8_to_jpeg(img1_temp)

        grayR= cv2.cvtColor(frameR,cv2.COLOR_BGR2GRAY)
        grayL= cv2.cvtColor(frameL,cv2.COLOR_BGR2GRAY)

        # Find the chess board corners
        retR, cornersR = cv2.findChessboardCorners(grayR,(cx,cy),None)
        retL, cornersL = cv2.findChessboardCorners(grayL,(cx,cy),None)

        # If corners are detected in left and right image then we save it.
        if (retR == True) and (retL == True) and timer <=0:
            count+=1
            cv2.imwrite(pathR+'img%d.png'%count,frameR)
            cv2.imwrite(pathL+'img%d.png'%count,frameL)

        if timer <=0:
            start = time.time()
except KeyboardInterrupt:
    print('Done ..')

In [ ]:
cameraL.stop()
cameraR.stop()

# Calibration using images

In [ ]:
print("Extracting image coordinates of respective 3D pattern ....\n")

# Termination criteria for refining the detected corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((cx*cy,3), np.float32)
objp[:,:2] = np.mgrid[0:cx,0:cy].T.reshape(-1,2)

img_ptsL = []
img_ptsR = []
obj_pts = []

nfiles = len(glob.glob(pathL+"img*.png"))
for i in range(1,nfiles+1):
    print(f"Reading img{i}.png ..")
    imgL = cv2.imread(pathL+"img%d.png"%i)
    imgR = cv2.imread(pathR+"img%d.png"%i)
    imgL_gray = cv2.imread(pathL+"img%d.png"%i,0)
    imgR_gray = cv2.imread(pathR+"img%d.png"%i,0)

    outputL = imgL.copy()
    outputR = imgR.copy()

    retR, cornersR =  cv2.findChessboardCorners(outputR,(cx,cy),None)
    retL, cornersL = cv2.findChessboardCorners(outputL,(cx,cy),None)

    if retR and retL:
        obj_pts.append(objp)
        cv2.cornerSubPix(imgR_gray,cornersR,(11,11),(-1,-1),criteria)
        cv2.cornerSubPix(imgL_gray,cornersL,(11,11),(-1,-1),criteria)
        cv2.drawChessboardCorners(outputR,(cx,cy),cornersR,retR)
        cv2.drawChessboardCorners(outputL,(cx,cy),cornersL,retL)
        cv2.imwrite(pathL+'/calib%d.png'%i,outputL)
        cv2.imwrite(pathR+'/calib%d.png'%i,outputR)

        img_ptsL.append(cornersL)
        img_ptsR.append(cornersR)


print("Calculating left camera parameters ... ")
# Calibrating left camera
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(obj_pts,img_ptsL,imgL_gray.shape[::-1],None,None)
hL,wL= imgL_gray.shape[:2]
new_mtxL, roiL= cv2.getOptimalNewCameraMatrix(mtxL,distL,(wL,hL),1,(wL,hL))

print("Calculating right camera parameters ... ")
# Calibrating right camera
retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(obj_pts,img_ptsR,imgR_gray.shape[::-1],None,None)
hR,wR= imgR_gray.shape[:2]
new_mtxR, roiR= cv2.getOptimalNewCameraMatrix(mtxR,distR,(wR,hR),1,(wR,hR))


print("Stereo calibration .....")
flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC
# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(obj_pts,
                                                          img_ptsL,
                                                          img_ptsR,
                                                          new_mtxL,
                                                          distL,
                                                          new_mtxR,
                                                          distR,
                                                          imgL_gray.shape[::-1],
                                                          criteria_stereo,
                                                          flags)

# Once we know the transformation between the two cameras we can perform stereo rectification
# StereoRectify function
rectify_scale= 1 # if 0 image croped, if 1 image not croped
rect_l, rect_r, proj_mat_l, proj_mat_r, Q, roiL, roiR= cv2.stereoRectify(new_mtxL, distL, new_mtxR, distR,
                                                 imgL_gray.shape[::-1], Rot, Trns,
                                                 rectify_scale,(0,0))

# Use the rotation matrixes for stereo rectification and camera intrinsics for undistorting the image
# Compute the rectification map (mapping between the original image pixels and 
# their transformed values after applying rectification and undistortion) for left and right camera frames
Left_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxL, distL, rect_l, proj_mat_l,
                                             imgL_gray.shape[::-1], cv2.CV_16SC2)
Right_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxR, distR, rect_r, proj_mat_r,
                                              imgR_gray.shape[::-1], cv2.CV_16SC2)


print("Saving parameters ......")
cv_file = cv2.FileStorage("data/params_py.xml", cv2.FILE_STORAGE_WRITE)
cv_file.write("Left_Stereo_Map_x",Left_Stereo_Map[0])
cv_file.write("Left_Stereo_Map_y",Left_Stereo_Map[1])
cv_file.write("Right_Stereo_Map_x",Right_Stereo_Map[0])
cv_file.write("Right_Stereo_Map_y",Right_Stereo_Map[1])
cv_file.release()
print("Done")

Plotting the calibration images ..

In [ ]:
nfiles = len(glob.glob(pathL+"calib*.png"))
imageWidgets = []

for i in range(1,nfiles+1):
    calL = cv2.imread(pathL+"calib%d.png"%i)
    calR = cv2.imread(pathR+"calib%d.png"%i)
    imageWidgets.append(widgets.HBox([widgets.Image(value=bgr8_to_jpeg(calL), width=410, height=308), 
                                      widgets.Image(value=bgr8_to_jpeg(calR), width=410, height=308)]))
    
display(widgets.VBox(imageWidgets))